In [1]:
import numpy as np 
import re 
import os 
import pandas as pd
from nltk.tokenize import RegexpTokenizer , sent_tokenize
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
import requests

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}
response = requests.get("https://example.com", headers=headers)
print(response.status_code)


200


In [3]:
stopWordsFile =     "C:/Users/sumit/Downloads/StopWords-20250923T151456Z-1-001/StopWords/StopWords_Auditor.txt"
positiveWordsFile = "C:/Users/sumit/Downloads/MasterDictionary-20250923T151506Z-1-001/MasterDictionary/positive-words.txt"
nagitiveWordsFile = "C:/Users/sumit/Downloads/MasterDictionary-20250923T151506Z-1-001/MasterDictionary/negative-words.txt"

In [4]:
input = pd.read_excel("C:/Users/sumit/Downloads/Input.xlsx")
input

def get_article_names(urls):
  titles = []
  for i in range (len(urls)):
    title = urls[i]
    title_clean = title[title.index( "m/" ) + 2 :-1]. replace('-' , ' ')
    titles.append(title_clean)
  return titles

urls =input["URL"]
urlsTitleDF = get_article_names(urls)
urlsTitleDF

['ai and ml based youtube analytics and content creation tool for optimizing subscriber engagement and content strategy',
 'enhancing front end features and functionality for improved user experience and dashboard accuracy in partner hospital application',
 'roas dashboard for campaign wise google ads budget tracking using google ads ap',
 'efficient processing and analysis of financial data from pdf files addressing formatting inconsistencies and ensuring data integrity for a toyota dealership management firm',
 'development of ea robot for automated trading',
 'ai and ml based youtube analytics and content creation tool for optimizing subscriber engagement and content strategy',
 'enhancing front end features and functionality for improved user experience and dashboard accuracy in partner hospital application',
 'roas dashboard for campaign wise google ads budget tracking using google ads ap',
 'efficient processing and analysis of financial data from pdf files addressing formatting 

In [5]:
url = "https://insights.blackcoffer.com/how-people-diverted-to-telehealth-services-and-telemedicine"

page=requests.get(url , headers={"User-Agent": "XY"})  
soup = BeautifulSoup(page.text , 'html.parser')
#get title
title = soup . find("h1",attrs = { 'class' : 'entry-title'}).get_text()

#get article text
text = soup . find(attrs = { 'class' : 'td-post-content'}).get_text()
# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

In [6]:
# Loading positive words
with open(positiveWordsFile,'r') as posfile:
    positivewords=posfile.read().lower()
positiveWordList=positivewords.split('\n')


# Loading negative words
with open(nagitiveWordsFile ,'r' ,  encoding="ISO-8859-1") as negfile:
    negativeword=negfile.read().lower()
negativeWordList=negativeword.split('\n')

#Loading stop words dictionary for removing stop words

with open(stopWordsFile ,'r') as stop_words:
    stopWords = stop_words.read().lower()
stopWordList = stopWords.split('\n')
stopWordList[-1:] = []

display( positiveWordList[:6]  , negativeWordList[:6] , stopWordList[:6])

['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable']

['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably']

['ernst', 'young', 'deloitte', 'touche', 'kpmg', 'pricewaterhousecoopers']

In [7]:
#tokenizeing module and filtering tokens using stop words list, removing punctuations
def tokenizer(text):
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopWordList, tokens))
    return filtered_words

def positive_score (text):
  posword=0
  tokenphrase = tokenizer(text)
  for word in tokenphrase :
    if word in positiveWordList:
       posword+=1
      
    retpos = posword
    return retpos 

def negative_score (text):
  negword=0
  tokenphrase = tokenizer(text)
  for word in tokenphrase :
    if word in negativeWordList : negword +=1

    retneg = negword 
    return retneg

def polarity_score (positive_score , negative_score) :
  return (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
#################################################
def total_word_count(text):
    tokens = tokenizer(text)
    return len(tokens)
#############################################
def AverageSentenceLenght (text):
  Wordcount = len(tokenizer (text))
  SentenceCount = len (sent_tokenize(text))
  if SentenceCount > 0 : Average_Sentence_Lenght = Wordcount / SentenceCount

  avg = Average_Sentence_Lenght

  return round(avg)


# Counting complex words
def complex_word_count(text):
    tokens = tokenizer(text)
    complexWord = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    return complexWord

def percentage_complex_word(text):
    tokens = tokenizer(text)
    complexWord = 0
    complex_word_percentage = 0
    
    for word in tokens:
        vowels=0
        if word.endswith(('es','ed')):
            pass
        else:
            for w in word:
                if(w=='a' or w=='e' or w=='i' or w=='o' or w=='u'):
                    vowels += 1
            if(vowels > 2):
                complexWord += 1
    if len(tokens) != 0:
        complex_word_percentage = complexWord/len(tokens)
    
    return complex_word_percentage

def fog_index(averageSentenceLength, percentageComplexWord):
    fogIndex = 0.4 * (averageSentenceLength + percentageComplexWord)
    return fogIndex

In [8]:
URLS = input ["URL"]
URLS

0      https://insights.blackcoffer.com/ai-and-ml-bas...
1      https://insights.blackcoffer.com/enhancing-fro...
2      https://insights.blackcoffer.com/roas-dashboar...
3      https://insights.blackcoffer.com/efficient-pro...
4      https://insights.blackcoffer.com/development-o...
                             ...                        
142    https://insights.blackcoffer.com/population-an...
143    https://insights.blackcoffer.com/google-lsa-ap...
144    https://insights.blackcoffer.com/healthcare-da...
145    https://insights.blackcoffer.com/budget-sales-...
146    https://insights.blackcoffer.com/amazon-buy-bo...
Name: URL, Length: 147, dtype: object

In [9]:
corps = []
for url in URLS:
    try:
        page = requests.get(url, headers={"User-Agent": "XY"}, timeout=20)
        soup = BeautifulSoup(page.text, 'html.parser')

        # get title safely
        title_tag = soup.find("h1", attrs={'class': 'entry-title'})
        title = title_tag.get_text(strip=True) if title_tag else "Title not found"

        # get article text safely
        content_tag = soup.find(attrs={'class': 'td-post-content'})
        if content_tag:
            lines = (line.strip() for line in content_tag.get_text().splitlines())
            chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
            text = '\n'.join(chunk for chunk in chunks if chunk)
        else:
            text = "Content not found"

        corps.append({"url": url, "title": title, "content": text})

    except Exception as e:
        print(f"Error scraping {url}: {e}")


In [13]:
#df = pd.DataFrame(corps,urlsTitleDF)
df = pd.DataFrame({'title':urlsTitleDF,'corps': corps})
df["total_word_count"] = df["corps"].apply(lambda x: total_word_count(x['content']))
df["percentage_complex_word"] = df["corps"].apply(lambda x: percentage_complex_word(x['content']))
df["complex_word_count"] = df["corps"].apply(lambda x: complex_word_count(x['content']))
df["AverageSentenceLenght"] = df["corps"].apply(lambda x: AverageSentenceLenght(x['content']))
df["positive_score"] = df["corps"].apply(lambda x: positive_score(x['content']))
df["negative_score"] = df["corps"].apply(lambda x: negative_score(x['content']))
df["polarity_score"] = np.vectorize(polarity_score)(df['positive_score'],df['negative_score'])

In [14]:
df

,title,corps,total_word_count,percentage_complex_word,complex_word_count,AverageSentenceLenght,positive_score,negative_score,polarity_score
0,ai and ml based youtube analytics and content ...,{'url': 'https://insights.blackcoffer.com/ai-a...,274,0.248175,68,46,0,0,0.0
1,enhancing front end features and functionality...,{'url': 'https://insights.blackcoffer.com/enha...,882,0.179138,158,24,0,0,0.0
2,roas dashboard for campaign wise google ads bu...,{'url': 'https://insights.blackcoffer.com/roas...,437,0.260870,114,24,0,0,0.0
3,efficient processing and analysis of financial...,{'url': 'https://insights.blackcoffer.com/effi...,578,0.354671,205,21,0,0,0.0
4,development of ea robot for automated trading,{'url': 'https://insights.blackcoffer.com/deve...,713,0.234222,167,17,0,0,0.0
...,...,...,...,...,...,...,...,...,...
142,population and community survey of america,{'url': 'https://insights.blackcoffer.com/popu...,919,0.218716,201,26,0,0,0.0
143,google lsa api data automation and dashboarding,{'url': 'https://insights.blackcoffer.com/goog...,1375,0.237818,327,21,0,0,0.0
144,healthcare data analysis,{'url': 'https://insights.blackcoffer.com/heal...,419,0.176611,74,30,0,0,0.0
145,budget sales kpi dashboard using power bi,{'url': 'https://insights.blackcoffer.com/budg...,106,0.198113,21,106,0,0,0.0


In [16]:
final = df.drop("corps", axis=1)
final

,title,total_word_count,percentage_complex_word,complex_word_count,AverageSentenceLenght,positive_score,negative_score,polarity_score
0,ai and ml based youtube analytics and content ...,274,0.248175,68,46,0,0,0.0
1,enhancing front end features and functionality...,882,0.179138,158,24,0,0,0.0
2,roas dashboard for campaign wise google ads bu...,437,0.260870,114,24,0,0,0.0
3,efficient processing and analysis of financial...,578,0.354671,205,21,0,0,0.0
4,development of ea robot for automated trading,713,0.234222,167,17,0,0,0.0
...,...,...,...,...,...,...,...,...
142,population and community survey of america,919,0.218716,201,26,0,0,0.0
143,google lsa api data automation and dashboarding,1375,0.237818,327,21,0,0,0.0
144,healthcare data analysis,419,0.176611,74,30,0,0,0.0
145,budget sales kpi dashboard using power bi,106,0.198113,21,106,0,0,0.0


In [18]:
final.to_csv('Output_Data_Structure.csv', index=False, encoding='utf-8')